<center><h1> CNN Keras: MNIST Dataset </h1></center>

Keras with TensorFlow as backend to calssify digits from the MNIST Dataset

In [6]:
!pip install keras
!pip install tensorflow

In [13]:
import numpy as np
from keras import backend as K
from keras.datasets import mnist
from keras.utils import np_utils

from keras.layers import Dense, Dropout,Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.models import Sequential
import pandas as pd

from sklearn.model_selection import train_test_split

## Modèle

In [21]:
model = Sequential()
K.set_image_data_format('channels_last')
np.random.seed(0)

In [47]:
X = pd.read_csv('./data/cnn/train.csv')
test = pd.read_csv('./data/cnn/test.csv')

y = X["label"]
X.drop(["label"], inplace = True, axis = 1)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2 , random_state=42)

In [54]:
X_train.shape

(33600, 28, 28, 1)

In [57]:
X_train[0].shape

(28, 28, 1)

## Data Eploration

In [16]:
import matplotlib.pyplot as plt
print("the number of training examples = %i" % X_train.shape[0])
print("the number of classes = %i" % len(numpy.unique(y_train)))
print("Dimention of images = {:d} x {:d}  ".format(X_train[1].shape[0],X_train[1].shape[1])  )

#This line will allow us to know the number of occurrences of each specific class in the data
unique, count= numpy.unique(y_train, return_counts=True)
print("The number of occuranc of each class in the dataset = %s " % dict (zip(unique, count) ), "\n" )
 
images_and_labels = list(zip(X_train,  y_train))
for index, (image, label) in enumerate(images_and_labels[:12]):
    plt.subplot(5, 4, index + 1)
    plt.axis('off')
    plt.imshow(image.squeeze(), cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('label: %i' % label )

the number of training examples = 33600
the number of classes = 10
Dimention of images = 28 x 28  
The number of occuranc of each class in the dataset = {0: 3316, 1: 3775, 2: 3331, 3: 3414, 4: 3233, 5: 3093, 6: 3352, 7: 3508, 8: 3228, 9: 3350}  



## Model Design and Achitecture

In [22]:
from keras.layers import Dropout

model.add(Conv2D(40, kernel_size=5, padding="same",input_shape=(28, 28, 1), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

In [23]:
model.add(Conv2D(70, kernel_size=3, padding="same", activation = 'relu'))
model.add(Conv2D(500, kernel_size=3, padding="same", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(1024, kernel_size=3, padding="valid", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

In [24]:
from keras.layers.core import Activation

model.add(Flatten())
model.add(Dense(units=100, activation='relu'  ))
model.add(Dropout(0.1))
model.add(Dense(units=100, activation='relu'  ))
model.add(Dropout(0.1))
model.add(Dense(units=100, activation='relu'  ))
model.add(Dropout(0.3))

model.add(Dense(10))
model.add(Activation("softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Evaluation

In [25]:
y_train = np_utils.to_categorical(y_train).astype('int32')
y_test = np_utils.to_categorical(y_test)

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

from keras.preprocessing.image import ImageDataGenerator
X_train2 = numpy.array(X_train, copy=True) 
y_train2 = numpy.array(y_train, copy=True) 

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    )

datagen.fit(X_train)

print(type(X_train2))
print(type(X_train))

# Concatenating the old data with the augmented data
result_x  = numpy.concatenate((X_train, X_train2), axis=0)
result_y  = numpy.concatenate((y_train, y_train2), axis=0)


# # fits the model on batches with real-time data augmentation:
history = model.fit_generator(datagen.flow(result_x, result_y, batch_size=35),
                    steps_per_epoch=len(X_train) / 32, epochs = 12)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Instructions for updating:
Use tf.cast instead.
Epoch 1/12
1050/1050 [==============================] - 375s 357ms/step - loss: 0.3778 - acc: 0.8805
Epoch 2/12
1050/1050 [==============================] - 372s 354ms/step - loss: 0.1197 - acc: 0.9688
Epoch 3/12
1050/1050 [==============================] - 372s 355ms/step - loss: 0.0993 - acc: 0.9750
Epoch 4/12
1050/1050 [==============================] - 373s 355ms/step - loss: 0.0882 - acc: 0.9793
Epoch 5/12
1050/1050 [==============================] - 377s 359ms/step - loss: 0.0752 - acc: 0.9813
Epoch 6/12
1050/1050 [==============================] - 376s 358ms/step - loss: 0.0710 - acc: 0.9833
Epoch 7/12
1050/1050 [==============================] - 374s 356ms/step - loss: 0.0685 - acc: 0.9844
Epoch 8/12
1050/1050 [==============================] - 373s 355ms/step - loss: 0.0571 - acc: 0.9867
Epoch 9/12
1050/1050 [==============================] - 373s 356ms/step - loss: 0.0684 - acc: 0.

In [27]:
scores = model.evaluate(X_test, y_test, verbose = 10 )
print ( scores )

[0.15181639777766773, 0.9901190476190476]


In [ ]:
test_set = (test.values).reshape(-1, 28, 28 , 1).astype('float32')

res = model.predict(test_set)
res = numpy.argmax(res,axis = 1)
res = pd.Series(res, name="Label")
submission = pd.concat([pd.Series(range(1 ,28001) ,name = "ImageId"),   res],axis = 1)
submission.to_csv("cnn_mnist_datagen.csv",index=False)
submission.head(10)